In [1]:
import pandas as pd
import datetime
import numpy as np
from myUtils import pickle_load, pickle_dump

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_colwidth', 18)
pd.set_option('display.width', 790)

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

fp_df_eval_sym_freq_results_train = 'df_eval_sym_freq_results_train'
fp_df_eval_sym_freq_results_validate = 'df_eval_sym_freq_results_validate'
fp_df_eval_sym_freq_results_test = 'df_eval_sym_freq_results_test'

str_today = datetime.date.today().strftime('%Y-%m-%d')
fp_results = f'yf_8models_results_{str_today}'

In [2]:
run_types = ['train', 'validate', 'test']

In [3]:
for run_type in run_types:
  if run_type == 'train':
    df = pickle_load(path_data_dump, fp_df_eval_sym_freq_results_train)
    name_table = 'tbl_train'
  elif run_type == 'validate':
    df = pickle_load(path_data_dump, fp_df_eval_sym_freq_results_validate)
    name_table = 'tbl_val'
  elif run_type == 'test':
    df = pickle_load(path_data_dump, fp_df_eval_sym_freq_results_test)
    name_table = 'tbl_test'
  else:
    msg_stop = f"ERROR run_type must be 'train', 'validate', or 'test', run_type is: {run_type}"
    raise SystemExit(msg_stop )

  tbl = df.groupby(['days_lookbacks', 'days_eval', 'sym_freq_cnt'])\
              .agg({'grp(CAGR)_mean':          ['count', 'mean', 'std',],
                    'grp(CAGR/UI)_mean':       ['mean', 'std',],
                    'grp(CAGR/retnStd)_mean':  ['mean', 'std',],
                    'SPY_CAGR':                ['mean', 'std',],
                    'SPY_CAGR/UI':             ['mean', 'std',],
                    'SPY_CAGR/retnStd':        ['mean', 'std',],                           
                    })
  
  tbl['dif_CAGR_mean']          = tbl['grp(CAGR)_mean', 'mean']         - tbl['SPY_CAGR', 'mean']
  tbl['dif_CAGR/UI_mean']       = tbl['grp(CAGR/UI)_mean', 'mean']      - tbl['SPY_CAGR/UI', 'mean'] 
  tbl['dif_CAGR/retnStd_mean']  = tbl['grp(CAGR/retnStd)_mean', 'mean'] - tbl['SPY_CAGR/retnStd', 'mean'] 
  tbl['grp_CAGR/UI_mean/std']   = tbl['grp(CAGR/UI)_mean', 'mean']      / tbl['grp(CAGR/UI)_mean', 'std']  
  tbl['SPY_CAGR/UI_mean/std']   = tbl['SPY_CAGR/UI', 'mean']            / tbl['SPY_CAGR/UI', 'std'] 
  tbl['dif_CAGR/UI_mean/std']   = tbl['grp_CAGR/UI_mean/std']           - tbl['SPY_CAGR/UI_mean/std']
  tbl['pct_rank_d_CAGR/UI_m/s'] = tbl['dif_CAGR/UI_mean/std'].rank(pct=True)
  tbl['run_type'] = run_type
  
  pickle_dump(tbl, path_data_dump, name_table)
  print(f'saved table for run_type: {run_type} to {path_data_dump}{name_table}') 


saved table for run_type: train to C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/tbl_train
saved table for run_type: validate to C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/tbl_val
saved table for run_type: test to C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/tbl_test


In [4]:
tbl_train = pickle_load(path_data_dump, 'tbl.train')
tbl_val = pickle_load(path_data_dump, 'tbl.validate')
tbl_test = pickle_load(path_data_dump, 'tbl.test')
print(f'len(tbl_train): {len(tbl_train)}')
print(f'len(tbl_val): {len(tbl_val)}')
print(f'len(tbl_test): {len(tbl_test)}')

len(tbl_train): 120
len(tbl_val): 117
len(tbl_test): 120


In [5]:
tbl_concat = pd.concat([tbl_train, tbl_val, tbl_test])
tbl_concat = tbl_concat.sort_values(by=["days_lookbacks", "days_eval", "sym_freq_cnt"])
print(f'len(tbl_concat): {len(tbl_concat)}')
tbl_concat.head()

len(tbl_concat): 357


grp(CAGR)_mean                             grp(CAGR/UI)_mean               grp(CAGR/retnStd)_mean                   SPY_CAGR                 SPY_CAGR/UI               SPY_CAGR/retnStd                dif_CAGR_mean dif_CAGR/UI_mean dif_CAGR/retnStd_mean grp_CAGR/UI_mean/std SPY_CAGR/UI_mean/std dif_CAGR/UI_mean/std pct_rank_d_CAGR/UI_m/s  run_type
                                               count          mean           std              mean           std                   mean           std         mean           std          mean           std             mean            std                                                                                                                                                     
days_lookbacks days_eval sym_freq_cnt                                                                                                                                                                                                                                                                                                                                                            
[120]          4         3                       400  2.712866e+30  5.425731e+31      1.225064e+31  2.450128e+32       3.592535e+30      7.140021e+31  1497.135745  29873.562552  2.990436e+09  5.974729e+10     48738.185444  963964.508874  2.712866e+30     1.225064e+31       3.592535e+30              0.050000             0.050051            -0.000051             0.833333         train
                         3                       114  3.244810e+04  2.244249e+05      5.967363e+10  4.484194e+11       5.554946e+05      3.815380e+06     2.452235      6.207977  3.018979e+06  1.204681e+07       898.773868    3171.147526  3.244565e+04     5.967062e+10       5.545958e+05              0.133076             0.250604            -0.117529             0.299145      validate
                         3                        57  5.934428e+02  2.509016e+03      9.730440e+08  4.689272e+09       2.177688e+04      6.700370e+04     3.749240     12.255535  5.020468e+06  2.292000e+07       443.248882    1600.900242  5.896936e+02     9.680235e+08       2.133364e+04              0.207504             0.219043            -0.011539             0.633333          test
               5         3                       400  2.981655e+36  4.211409e+37      1.489522e+37  2.103861e+38       4.497894e+36      6.353007e+37     4.602147     66.528097  8.102190e+06  1.483189e+08       345.600477    2706.665766  2.981655e+36     1.489522e+37       4.497894e+36              0.070799             0.054627             0.016173             0.941667         train
                         3                       114  2.935490e+03  1.295024e+04      4.269935e+08  2.600548e+09       4.899959e+04      2.159221e+05     0.979413      2.046460  1.463127e+03  3.110262e+03       120.667237     241.474615  2.934511e+03     4.269921e+08       4.887892e+04              0.164194             0.470419            -0.306226             0.051282      validate

In [6]:
tbl_concat.loc['[30, 60, 120]', [4], [9]]


grp(CAGR)_mean                               grp(CAGR/UI)_mean               grp(CAGR/retnStd)_mean                   SPY_CAGR                 SPY_CAGR/UI               SPY_CAGR/retnStd                dif_CAGR_mean dif_CAGR/UI_mean dif_CAGR/retnStd_mean grp_CAGR/UI_mean/std SPY_CAGR/UI_mean/std dif_CAGR/UI_mean/std pct_rank_d_CAGR/UI_m/s  run_type
                                               count           mean            std              mean           std                   mean           std         mean           std          mean           std             mean           std                                                                                                                                                      
days_lookbacks days_eval sym_freq_cnt                                                                                                                                                                                                                                                                                                                                                              
[30, 60, 120]  4         9                       128  140571.381177  979721.046331      2.674217e+11  1.958404e+12       8.855914e+06      6.661397e+07  4701.937810  52807.975285  9.401742e+09  1.056161e+11    152256.166211  1.704007e+06  135869.443366     2.580199e+11       8.703658e+06              0.136551             0.089018             0.047533             0.983333         train
                         9                        10       0.008514       0.007328      4.542006e+01  3.909137e+01       2.987081e+01      2.570870e+01     5.232702      4.731019  9.466733e+03  9.060360e+03       685.132589  5.385016e+02      -5.224188    -9.421313e+03      -6.552618e+02              1.161895             1.044852             0.117043             0.829060      validate
                         9                        21      42.652255     113.022125      5.058816e+07  2.238615e+08       1.976892e+03      5.598073e+03     4.366096     11.145372  4.175305e+06  1.910487e+07       633.568122  1.961256e+03      38.286160     4.641286e+07       1.343324e+03              0.225980             0.218547             0.007433             0.733333          test

In [7]:
# change run_type train, validate, test to 100, 10, 1 to apply aggregate functions
run_type_values = [100, 10, 1]
z_run_type_values = zip(run_types, run_type_values)

for run_type, value in z_run_type_values:
  tbl_concat.loc[tbl_concat['run_type'] == run_type, 'run_type'] = value

In [8]:
print(f'len(tbl_concat): {len(tbl_concat)}')
tbl_concat.head()

len(tbl_concat): 357


grp(CAGR)_mean                             grp(CAGR/UI)_mean               grp(CAGR/retnStd)_mean                   SPY_CAGR                 SPY_CAGR/UI               SPY_CAGR/retnStd                dif_CAGR_mean dif_CAGR/UI_mean dif_CAGR/retnStd_mean grp_CAGR/UI_mean/std SPY_CAGR/UI_mean/std dif_CAGR/UI_mean/std pct_rank_d_CAGR/UI_m/s run_type
                                               count          mean           std              mean           std                   mean           std         mean           std          mean           std             mean            std                                                                                                                                                    
days_lookbacks days_eval sym_freq_cnt                                                                                                                                                                                                                                                                                                                                                           
[120]          4         3                       400  2.712866e+30  5.425731e+31      1.225064e+31  2.450128e+32       3.592535e+30      7.140021e+31  1497.135745  29873.562552  2.990436e+09  5.974729e+10     48738.185444  963964.508874  2.712866e+30     1.225064e+31       3.592535e+30              0.050000             0.050051            -0.000051             0.833333          100
                         3                       114  3.244810e+04  2.244249e+05      5.967363e+10  4.484194e+11       5.554946e+05      3.815380e+06     2.452235      6.207977  3.018979e+06  1.204681e+07       898.773868    3171.147526  3.244565e+04     5.967062e+10       5.545958e+05              0.133076             0.250604            -0.117529             0.299145           10
                         3                        57  5.934428e+02  2.509016e+03      9.730440e+08  4.689272e+09       2.177688e+04      6.700370e+04     3.749240     12.255535  5.020468e+06  2.292000e+07       443.248882    1600.900242  5.896936e+02     9.680235e+08       2.133364e+04              0.207504             0.219043            -0.011539             0.633333            1
               5         3                       400  2.981655e+36  4.211409e+37      1.489522e+37  2.103861e+38       4.497894e+36      6.353007e+37     4.602147     66.528097  8.102190e+06  1.483189e+08       345.600477    2706.665766  2.981655e+36     1.489522e+37       4.497894e+36              0.070799             0.054627             0.016173             0.941667          100
                         3                       114  2.935490e+03  1.295024e+04      4.269935e+08  2.600548e+09       4.899959e+04      2.159221e+05     0.979413      2.046460  1.463127e+03  3.110262e+03       120.667237     241.474615  2.934511e+03     4.269921e+08       4.887892e+04              0.164194             0.470419            -0.306226             0.051282           10

#### These are the best performing run parameters for days_lookbacks, days_eval, sym_freq_cnt. The performance metric is "pct_rank_d_CAGR/UI_m/s". The metric first calculate the difference between the group's "CAGR/UI_mean/CAGR/UI_std" and the SPY's "CAGR/UI_mean/CAGR/UI_std" for run_type "train", "validate" and "test". It does a percentile rank of the difference for each run_type. The hightest ranking is 1. For a set of run parameters (i.e. days_lookbacks, days_eval, sym_freq_cnt), the sum of "pct_rank_d_CAGR/UI_m/s" for the three run_types is 3 maximum. In which case, that set of run parameters has performed the best in all three run_types (i.e. ranked 1 in each run_type).

#### The column "run_type sum" shows run_types in calculated the result for that row.  For example, a "run_type sum" of 111 means that run_type train, validate and test were used in the calculation, since train=100, validate=10, test=1, the sum is 111.   

In [9]:
df_agg = tbl_concat.groupby(['days_lookbacks', 'days_eval', 'sym_freq_cnt']).agg(['min', 'max', 'sum'])
print(f'len(df_agg): {len(df_agg)}')

# lowest risk since we are picking from the best of the worst, i.e. sorting on 'min'
yf_8models_dif_min = df_agg.sort_values(by=[('dif_CAGR/UI_mean/std', '', 'min')], ascending=False)

# highest percentile ranking across test, validate and test datasets
yf_8models_pct_rnk = df_agg.sort_values(by=[('pct_rank_d_CAGR/UI_m/s', '', 'sum')], ascending=False)


########## Select a model to display from below: ##########
print(f'Lowest Risk Model: yf_8models_dif_min')
yf_8models_dif_min.iloc[:, np.r_[2:4, 39:63]].head(8)  # lowest risk

# print(f'Highest Rank Model: yf_8models_pct_rnk')
# yf_8models_pct_rnk.iloc[:, np.r_[2:4, 39:63]].head(8)  # highest ranks of combined datasets

len(df_agg): 120
Lowest Risk Model: yf_8models_dif_min


grp(CAGR)_mean               dif_CAGR_mean                             dif_CAGR/UI_mean                             dif_CAGR/retnStd_mean                             grp_CAGR/UI_mean/std                     SPY_CAGR/UI_mean/std                     dif_CAGR/UI_mean/std                     pct_rank_d_CAGR/UI_m/s                     run_type          
                                                  count          mean                                                                                                                                                                                                                                                                                                                                  
                                                    sum           min           min           max           sum              min           max           sum                   min           max           sum                  min       max       sum                  min       max       sum                  min       max       sum                    min       max       sum      min  max  sum
days_lookbacks    days_eval sym_freq_cnt                                                                                                                                                                                                                                                                                                                                                               
[30, 60, 120]     4         9                       159  8.513929e-03 -5.224188e+00  1.358694e+05  1.359025e+05    -9.421313e+03  2.580199e+11  2.580663e+11      -6.552618e+02     8.703658e+06  8.704346e+06           0.136551    1.161895  1.524426           0.089018    1.044852  1.352416           0.007433    0.117043  0.172009           0.733333      0.983333  2.545726        1  100  111
[15, 30, 60, 120] 5         12                       30  1.314221e+02  1.299006e+02  1.299449e+04  1.312439e+04     1.881279e+04  2.894501e+10  2.894503e+10       4.573625e+03     9.753671e+05  9.799407e+05           0.209459    0.590067  0.799525           0.208579    0.556826  0.765406           0.000879    0.033240  0.034120           0.816667      0.883333  1.700000        1  100  101
[30, 60, 120]     4         8                       287  4.728288e-02 -2.910465e+00  2.251341e+31  2.251341e+31    -2.838186e+05  1.016651e+32  1.016651e+32      -9.258968e+01     2.981200e+31  2.981200e+31           0.064416    0.982400  1.460210           0.064910    0.351728  0.712894          -0.000495    0.686145  0.747316           0.783333      0.965812  2.674145        1  100  111
[15, 30, 60, 120] 5         7                       454  7.049188e-02 -9.731311e-01  2.210351e+54  2.210351e+54     4.316819e+04  4.942494e+60  4.942494e+60       5.652030e+01     5.038295e+54  5.038295e+54           0.055728    0.845146  1.109533           0.059758    0.338482  0.546658          -0.004031    0.506664  0.562875           0.733333      0.957265  2.598932        1  100  111
[30, 60, 120]     6         9                       168  3.128783e-02 -4.359894e+00  1.571663e+04  1.572918e+04    -8.193669e+03  5.239630e+08  5.522375e+08      -5.346872e+02     2.029231e+05  2.032941e+05           0.087336    2.294777  2.647089           0.093635    1.948489  2.278452          -0.006300    0.346288  0.368637           0.691667      0.914530  2.414530        1  100  111
[15, 30, 60, 120] 4         5                       568  1.052705e+04  1.052549e+04  6.854585e+26  6.854585e+26     2.077643e+10  6.192848e+28  6.192848e+28       2.100606e+05     1.464170e+27  1.464170e+27           0.071883    0.255270  0.560918           0.050561    0.244022  0.503351          -0.010256    0.046502  0.057567           0.650000      0.966667  2.368803        1  100  111
[30, 60, 120]     4         3                       566  1.020422e+07  1.020422e+07  1.545655e+20  1.545655e+20     2.040

In [10]:
print(f'Highest Rank Model: yf_8models_pct_rnk')
yf_8models_pct_rnk.iloc[:, np.r_[2:4, 39:63]].head(8)  # highest ranks of combined datasets

Highest Rank Model: yf_8models_pct_rnk


grp(CAGR)_mean               dif_CAGR_mean                             dif_CAGR/UI_mean                             dif_CAGR/retnStd_mean                             grp_CAGR/UI_mean/std                     SPY_CAGR/UI_mean/std                     dif_CAGR/UI_mean/std                     pct_rank_d_CAGR/UI_m/s                     run_type          
                                                  count          mean                                                                                                                                                                                                                                                                                                                                  
                                                    sum           min           min           max           sum              min           max           sum                   min           max           sum                  min       max       sum                  min       max       sum                  min       max       sum                    min       max       sum      min  max  sum
days_lookbacks    days_eval sym_freq_cnt                                                                                                                                                                                                                                                                                                                                                               
[30, 60, 120]     4         8                       287      0.047283     -2.910465  2.251341e+31  2.251341e+31    -2.838186e+05  1.016651e+32  1.016651e+32         -92.589679     2.981200e+31  2.981200e+31           0.064416    0.982400  1.460210           0.064910    0.351728  0.712894          -0.000495    0.686145  0.747316           0.783333      0.965812  2.674145        1  100  111
[15, 30, 60, 120] 5         7                       454      0.070492     -0.973131  2.210351e+54  2.210351e+54     4.316819e+04  4.942494e+60  4.942494e+60          56.520301     5.038295e+54  5.038295e+54           0.055728    0.845146  1.109533           0.059758    0.338482  0.546658          -0.004031    0.506664  0.562875           0.733333      0.957265  2.598932        1  100  111
[30, 60, 120]     4         9                       159      0.008514     -5.224188  1.358694e+05  1.359025e+05    -9.421313e+03  2.580199e+11  2.580663e+11        -655.261774     8.703658e+06  8.704346e+06           0.136551    1.161895  1.524426           0.089018    1.044852  1.352416           0.007433    0.117043  0.172009           0.733333      0.983333  2.545726        1  100  111
                            4                       561  17525.743927  17523.642497  3.010245e+28  3.010245e+28     1.771513e+07  2.688766e+30  2.688766e+30      361731.875094     6.552447e+28  6.552447e+28           0.070799    0.539416  0.799411           0.050394    0.231028  0.495010          -0.024393    0.308388  0.304401           0.591667      0.958333  2.447436        1  100  111
                  6         9                       168      0.031288     -4.359894  1.571663e+04  1.572918e+04    -8.193669e+03  5.239630e+08  5.522375e+08        -534.687225     2.029231e+05  2.032941e+05           0.087336    2.294777  2.647089           0.093635    1.948489  2.278452          -0.006300    0.346288  0.368637           0.691667      0.914530  2.414530        1  100  111
[15, 30, 60, 120] 4         5                       568  10527.048605  10525.489495  6.854585e+26  6.854585e+26     2.077643e+10  6.192848e+28  6.192848e+28      210060.559134     1.464170e+27  1.464170e+27           0.071883    0.255270  0.560918           0.050561    0.244022  0.503351          -0.010256    0.046502  0.057567           0.650000      0.966667  2.368803        1  100  111
[30, 60, 120]     5         7                       345      0.044418     -0.516384  5.527670e+10  5.527698e+10     6.727

In [11]:
# # select by days_lookbacks, days_eval, sym_freq_cnt
# tbl_concat.loc['[15, 30, 60, 120]', 4, 9]